In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time
import random

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC

NUM_OF_ROWS = 9000
ATTACK_NAME = 'DoS'

In [2]:
pd.set_option('display.max_columns', None)  # Show all columns

---

In [3]:
# import the attack sample dataset
dosSamples = pd.read_csv('dos_goldeneye_samples.csv') 
dosSamples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,3,515.344252,50,1282,462.681723,214074.376414,1402323,1248,535.646677,18,477.240416,266,154.531034,16,84.645472,0.0,0.0,590.451789,0,0,0,40.210026,75.926337,0.802339,0.013175,0.035396
1,1,115.500000,100,131,15.500000,240.250000,246,77,61.500000,46,15.500000,0,0.000000,0,0.000000,0.0,0.0,123.000000,0,0,0,8.514560,0.469784,8.514545,2.838187,4.013792
2,0,115.000000,115,115,0.000000,0.000000,0,0,0.000000,0,0.000000,81,81.000000,81,0.000000,0.0,0.0,0.000000,0,0,0,39.116539,0.997021,2.048092,1.029383,0.172540
3,1,98.500000,94,103,4.500000,20.250000,840,40,40.000000,40,0.000000,49,49.000000,49,0.000000,0.0,0.0,42.000000,0,0,0,37.484924,1.120450,2.009573,0.914266,0.938558
4,0,101.000000,101,101,0.000000,0.000000,0,0,0.000000,0,0.000000,47,47.000000,47,0.000000,0.0,0.0,0.000000,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,1,143.000000,143,143,0.000000,0.000000,218,109,109.000000,109,0.000000,0,0.000000,0,0.000000,0.0,0.0,218.000000,0,0,0,8.499306,0.235313,8.499306,8.499306,0.000000
324,2,98.166667,93,103,4.844814,23.472222,118,40,39.333333,39,0.471405,49,49.000000,49,0.000000,0.0,0.0,59.000000,0,0,0,8.036513,0.746592,6.632294,1.607303,2.538397
325,1,160.500000,66,548,174.403507,30416.583333,612,514,204.000000,32,219.642133,83,49.000000,32,24.041631,172.0,17.0,0.000000,0,6,0,0.163065,36.795153,0.066440,0.032613,0.029020
326,1,127.500000,92,163,35.500000,1260.250000,174,58,58.000000,58,0.000000,129,129.000000,129,0.000000,0.0,0.0,0.000000,0,0,0,0.005676,1057.076658,0.004388,0.001135,0.001652


In [4]:
dosSamples = dosSamples[dosSamples['ACK Flag Count'] > 100]

In [5]:
# print some general information about the attack samples
print(f'Dataset Shape: {dosSamples.shape}')
dosSamples.reset_index() 

Dataset Shape: (21, 26)


,index,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,26,1,124.775482,66,845,145.078895,21047.885732,584231,811,90.775482,32,145.078895,0,0.0,0,0.0,55.619484,0.0,117.268366,2539,3897,0,5.743676,1120.536707,0.023122,0.000893,0.001892
1,27,1,121.000527,66,849,140.240047,19667.270800,825026,815,87.000527,32,140.240047,0,0.0,0,0.0,52.001476,0.0,97.084726,3555,5928,0,10.782026,879.519319,0.029030,0.001137,0.002449
2,40,1,122.496462,66,860,141.830236,20115.815856,850451,826,88.496462,32,141.830236,0,0.0,0,0.0,53.694381,0.0,96.074446,3366,6244,0,11.796613,814.640611,0.078117,0.001228,0.002738
3,51,1,122.480506,66,829,142.044040,20176.509262,839680,795,88.480506,32,142.044040,0,0.0,0,0.0,53.649737,0.0,95.083230,3358,6132,0,12.465338,761.311086,0.051932,0.001314,0.002873
4,69,1,123.573296,66,831,143.762818,20667.747896,863397,797,89.573296,32,143.762818,0,0.0,0,0.0,54.859322,0.0,96.221665,3270,6369,0,12.742510,756.444383,0.061831,0.001322,0.002859
5,82,1,123.609547,66,881,143.952884,20722.432703,856040,847,89.609547,32,143.952884,0,0.0,0,0.0,54.859416,0.0,97.199955,3284,6269,0,14.330901,666.601486,0.063931,0.001500,0.003420
6,91,1,104.405674,54,833,120.419451,14500.844118,672515,799,70.405674,20,120.419451,0,0.0,0,0.0,36.545959,0.0,74.607832,2519,6834,3074,11.847213,806.265573,3.898364,0.001240,0.040068
7,106,1,109.580024,54,792,126.620150,16032.662369,690877,758,75.580024,20,126.620150,0,0.0,0,0.0,41.594246,0.0,82.345292,2911,5802,1722,14.243538,641.764701,5.236684,0.001558,0.054854
8,124,1,122.578599,66,813,141.961046,20152.938582,802965,779,88.578599,32,141.961046,0,0.0,0,0.0,53.393602,0.0,105.932058,3609,5456,0,8.945775,1013.327517,0.137966,0.000987,0.003047
9,136,1,106.246133,54,911,122.568377,15023.007152,541846,877,72.246133,20,122.568377,0,0.0,0,0.0,38.640800,0.0,81.005531,1871,5385,1990,7.606195,986.038358,0.115753,0.001014,0.003427


In [6]:
# find the columns that we need to synthesis data for to produce an attack dataset
columnsToGather = dosSamples.replace(0, np.nan) #replace all 0 values with null
columnsToGather = columnsToGather.dropna(how="all", axis=1).columns.tolist() #remove all columns where there are null values
columnsToGather #left with all columns that the values are not 0 (be know for a fact that the data is consistant and there are not missing values in the rows)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Packet Length Std',
 'Packet Length Variance',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Packet Length Std',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'ACK Flag Count',
 'RST Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

In [7]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
RST_FlagCount = dosSamples[dosSamples['RST Flag Count'] != 0]['RST Flag Count']

MinMaxDict = {col: (dosSamples[col].min() * 0.85, dosSamples[col].max() * 1.15) for col in columnsToGather}
MinMaxDict['Number of Ports'] = (1, 1)
MinMaxDict['RST Flag Count'] = (np.min(RST_FlagCount), np.max(RST_FlagCount))
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(88.74482307370185),
  np.float64(146.11285425101212)),
 'Packet Length Min': (np.float64(45.9), np.float64(75.89999999999999)),
 'Packet Length Max': (np.float64(673.1999999999999),
  np.float64(1047.6499999999999)),
 'Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Packet Length Variance': (np.float64(12325.7175000617),
  np.float64(25000.66097440541)),
 'Total Length of Fwd Packet': (np.float64(378120.8),
  np.float64(1004422.6499999999)),
 'Fwd Packet Length Max': (np.float64(644.3), np.float64(1008.55)),
 'Fwd Packet Length Mean': (np.float64(59.84482307370184),
  np.float64(107.01285425101214)),
 'Fwd Packet Length Min': (np.float64(17.0), np.float64(36.8)),
 'Fwd Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Fwd Segment Size Avg': (np.float64(31.06406511725293),
  np.float64(67.50656829320263)),
 'Subflow Fwd Bytes': (np.float64(63.

In [8]:
# change values to int for the columns that should be int (because when calculating approximate values they will be turned into floats)
intColumns = ['SYN Flag Count', 'ACK Flag Count', 'RST Flag Count', 'Packet Length Min', 'Packet Length Max', 'Total Length of Fwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Bwd Packet Length Max', 'Bwd Packet Length Min']
for key, val in MinMaxDict.items():
    if key in intColumns:
        MinMaxDict[key] = tuple([int(v) for v in val])
MinMaxDict

{'Number of Ports': (1, 1),
 'Average Packet Length': (np.float64(88.74482307370185),
  np.float64(146.11285425101212)),
 'Packet Length Min': (45, 75),
 'Packet Length Max': (673, 1047),
 'Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Packet Length Variance': (np.float64(12325.7175000617),
  np.float64(25000.66097440541)),
 'Total Length of Fwd Packet': (378120, 1004422),
 'Fwd Packet Length Max': (644, 1008),
 'Fwd Packet Length Mean': (np.float64(59.84482307370184),
  np.float64(107.01285425101214)),
 'Fwd Packet Length Min': (17, 36),
 'Fwd Packet Length Std': (np.float64(102.35653313322236),
  np.float64(169.5604910365803)),
 'Fwd Segment Size Avg': (np.float64(31.06406511725293),
  np.float64(67.50656829320263)),
 'Subflow Fwd Bytes': (np.float64(63.41665742178833),
  np.float64(134.85862103572862)),
 'SYN Flag Count': (1489, 4242),
 'ACK Flag Count': (3211, 7859),
 'RST Flag Count': (95, 3074),
 'Flow Duration': (np.float64(4.8821247577

### Creating the dataset

In [9]:
# creating an empty dataframe before adding values to it
dosDataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dosSamples.columns))), columns=dosSamples.columns)
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# adding zeros to all columns that should not have any values
zeroColumns = [col for col in dosSamples.columns if col not in columnsToGather]
for col in zeroColumns:
    dosDataset[col] = int(0)
zeroColumns

['Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg']

In [11]:
dosDataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### First group

In [12]:
# finding the correlation between the SYN Flag Count column to the rest of the columns in order to create new data
firstCorrelation = ['SYN Flag Count', 'ACK Flag Count']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dosSamples[firstCorrelation[1]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('ACK Flag Count', np.float64(1.8289174466546883))


In [13]:
dosDataset['SYN Flag Count'] = np.random.randint(MinMaxDict['SYN Flag Count'][0], MinMaxDict['SYN Flag Count'][1], NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = int(row['SYN Flag Count'] * updatedFactor)

In [14]:
dosDataset.head(10)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3577,5735.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1650,2562.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1642,3531.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2684,4242.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2201,4570.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3520,5394.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2972,5988.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2567,3920.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2490,5309.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2504,3881.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# adding the RST flag such that half of the rows will have 0 and the other half will have a random value from the known range
midpoint = len(dosDataset) // 2 #calculate the midpoint of the DataFrame

# Generate random integers for the second half
random_values = np.random.randint(MinMaxDict['RST Flag Count'][0], MinMaxDict['RST Flag Count'][1] + 1, size=len(dosDataset) - midpoint)

# Add the new column with 0s for the first half and random integers for the second half
new_column_values = [0] * midpoint + list(random_values)
dosDataset['RST Flag Count'] = new_column_values

In [16]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3577,5735.0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1650,2562.0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1642,3531.0,0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2684,4242.0,0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2201,4570.0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2195,4573.0,2947,0.0,0.0,0.0,0.0,0.0
8996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3657,7733.0,414,0.0,0.0,0.0,0.0,0.0
8997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1570,2557.0,2947,0.0,0.0,0.0,0.0,0.0
8998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1695,3492.0,3032,0.0,0.0,0.0,0.0,0.0


### Second group

In [17]:
# Generate random values for the 'Flow Duration' column
randValues = np.random.uniform(MinMaxDict['Flow Duration'][0]*0.9, MinMaxDict['Flow Duration'][1]*1.05, size=NUM_OF_ROWS)

# Assign the random values
dosDataset['Flow Duration'] = randValues

In [18]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3577,5735.0,0,7.561949,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1650,2562.0,0,12.779367,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1642,3531.0,0,13.117692,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2684,4242.0,0,23.037182,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2201,4570.0,0,19.756791,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2195,4573.0,2947,21.838815,0.0,0.0,0.0,0.0
8996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3657,7733.0,414,35.557013,0.0,0.0,0.0,0.0
8997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1570,2557.0,2947,20.488110,0.0,0.0,0.0,0.0
8998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1695,3492.0,3032,22.115640,0.0,0.0,0.0,0.0


In [19]:
# finding the correlation between the Number of Ports column to the rest of the columns in order to create new data
secondCorrelation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dosSamples[secondCorrelation[0]].values.reshape(-1, 1) #column 'Number of Ports'
dependent_cols = dosSamples[secondCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(secondCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packets Per Second', np.float64(30.145241232971678))
('IAT Max', np.float64(0.3696369152684689))
('IAT Mean', np.float64(0.00011623768083943193))
('IAT Std', np.float64(0.0041811844462341175))


In [20]:
durationToPacketsCorr = [x * y for x, y in zip(dosSamples['Flow Duration'].values, dosSamples['Packets Per Second'].values)]
durationToPacketsCorr = np.mean(durationToPacketsCorr)
durationToPacketsCorr

np.float64(8732.047619047618)

In [21]:
#iterating over all rows we need to add values
for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: 
        # calculate a random small delta of the factor for adding some randomness
        if col == 'Packets Per Second':
            delta = random.uniform(durationToPacketsCorr * 0.1, durationToPacketsCorr * 0.2) # select a delta
            updatedFactor = durationToPacketsCorr + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std' or col == 'IAT Max':
                delta = random.uniform(factor * 0.7, factor * 0.99)
                updatedFactor = factor + random.choices([-1, 1], weights=[2, 1], k=1)[0] * delta  
            else:
                delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dosDataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [22]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.0,21.0,21.0,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,1.0,118.162351,62.000000,835.333333,137.140846,18874.235571,737972.047619,801.333333,84.162351,28.000000,137.140846,0.0,0.0,0.0,0.0,49.548964,0.0,95.038656,3010.761905,5627.904762,562.238095,15.224132,700.470398,4.264842,0.001767,0.049179
std,0.0,6.975820,5.796551,29.169048,8.363914,2241.856330,128170.354837,29.169048,6.975820,5.796551,8.363914,0.0,0.0,0.0,0.0,6.637618,0.0,9.910132,551.209389,852.917400,897.206103,7.796555,276.305968,6.595806,0.000968,0.074095
min,1.0,104.405674,54.000000,792.000000,120.419451,14500.844118,444848.000000,758.000000,70.405674,20.000000,120.419451,0.0,0.0,0.0,0.0,36.545959,0.0,74.607832,1752.000000,3778.000000,0.000000,5.743676,246.263767,0.023122,0.000893,0.001892
25%,1.0,109.976270,54.000000,818.000000,127.842838,16343.791319,640328.000000,784.000000,75.976270,20.000000,127.842838,0.0,0.0,0.0,0.0,41.808258,0.0,88.539871,2656.000000,5385.000000,0.000000,9.452635,457.067517,0.063931,0.001061,0.002862
50%,1.0,121.000527,66.000000,831.000000,140.240047,19667.270800,773385.000000,797.000000,87.000527,32.000000,140.240047,0.0,0.0,0.0,0.0,52.030510,0.0,96.074446,3234.000000,5802.000000,0.000000,12.632054,756.444383,0.137966,0.001322,0.003427
75%,1.0,123.044975,66.000000,849.000000,142.734482,20373.132473,846118.000000,815.000000,89.044975,32.000000,142.734482,0.0,0.0,0.0,0.0,53.888530,0.0,101.706229,3366.000000,6244.000000,1035.000000,19.535407,943.017495,5.236684,0.002188,0.054854
max,1.0,127.054656,66.000000,911.000000,147.443905,21739.705195,873411.000000,877.000000,93.054656,32.000000,147.443905,0.0,0.0,0.0,0.0,58.701364,0.0,117.268366,3689.000000,6834.000000,3074.000000,34.094212,1120.536707,20.601452,0.004061,0.230153


In [23]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2861.120222,5229.870778,787.155667,22.608102,532.919539,6.006740,0.002629,0.067981
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,784.638051,1651.722496,994.274012,10.489316,385.614713,7.910678,0.001302,0.089145
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1489.000000,2197.000000,0.000000,4.394261,170.041244,0.019663,0.000414,0.000209
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2187.750000,3913.000000,0.000000,13.547278,276.940191,0.741335,0.001531,0.008365
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2853.000000,5084.000000,47.500000,22.625177,386.616706,1.941433,0.002562,0.021624
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3535.000000,6348.250000,1572.000000,31.544778,638.536413,9.085513,0.003606,0.104833
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4241.000000,9273.000000,3074.000000,41.166914,2321.145044,30.111836,0.005715,0.341234


In [24]:
x = dosDataset[dosDataset['Flow Duration']<6]
x[x['Flow Duration']>5][0:40]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2334,4837.0,0,5.277490,1424.866466,3.439390,0.000542,0.001817
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3692,7855.0,0,5.455365,1360.572878,0.149390,0.000535,0.003586
87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2685,4159.0,0,5.886212,1661.191723,0.189224,0.000771,0.043958
90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3594,7661.0,0,5.026781,1514.560718,0.333893,0.000668,0.037344
143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3235,6637.0,0,5.460268,1788.527274,3.836920,0.000517,0.042578
153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1511,2475.0,0,5.821715,1657.480429,0.113293,0.000558,0.001294
154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2116,4587.0,0,5.913239,1185.072759,3.814854,0.000590,0.004707
158,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2428,3931.0,0,5.443123,1340.919767,0.098407,0.000738,0.044134
172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3247,6538.0,0,5.535059,1302.795527,0.400006,0.000542,0.044067
186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2669,3981.0,0,5.431334,1899.736168,0.554891,0.000757,0.001099


In [25]:
x = dosDataset[dosDataset['Flow Duration']<50.5]
x[x['Flow Duration']>25][20:40]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,1604,2396.0,0,26.508911,391.809502,17.415553,0.003630,0.198255
51,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2151,4519.0,0,33.581932,212.456292,3.440369,0.004385,0.040329
53,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3229,5204.0,0,26.408210,392.120667,19.160479,0.003621,0.005870
55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3276,5108.0,0,28.515070,257.216652,0.476622,0.002789,0.002516
57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2229,4694.0,0,39.098097,255.998620,0.611397,0.003943,0.301529
58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,4240,6951.0,0,36.077246,197.824392,3.556051,0.003653,0.042832
65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2324,4782.0,0,37.166575,276.766913,1.750534,0.004843,0.014178
66,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2848,4409.0,0,38.372818,202.898697,2.551154,0.003805,0.047882
67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3665,7862.0,0,29.266726,338.648584,0.995675,0.002790,0.011360
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2232,4835.0,0,30.008803,336.519746,20.837997,0.003876,0.026832


In [26]:
dosDataset[dosDataset['Flow Duration']<6][:20]

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2551,4180.0,0,4.539432,1648.346631,0.122265,0.000619,0.001557
33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2334,4837.0,0,5.277490,1424.866466,3.439390,0.000542,0.001817
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3692,7855.0,0,5.455365,1360.572878,0.149390,0.000535,0.003586
64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2471,5312.0,0,4.851205,1990.216674,0.130352,0.000628,0.001120
87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2685,4159.0,0,5.886212,1661.191723,0.189224,0.000771,0.043958
90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3594,7661.0,0,5.026781,1514.560718,0.333893,0.000668,0.037344
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2040,4128.0,0,4.663438,1545.379447,0.276510,0.000437,0.001167
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,2874,4699.0,0,4.394261,1620.298518,0.120075,0.000444,0.001253
136,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3620,7285.0,0,4.438264,1627.072177,0.261944,0.000599,0.031949
143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,3235,6637.0,0,5.460268,1788.527274,3.836920,0.000517,0.042578


### Third group

In [27]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
firstCorrelation = ['Average Packet Length', 'Packet Length Std', 'Packet Length Variance', 'Total Length of Fwd Packet', 
                    'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Fwd Segment Size Avg', 'Subflow Fwd Bytes']
independent_col = dosSamples[firstCorrelation[0]].values.reshape(-1, 1) #column 'Packet Length Std'
dependent_cols = dosSamples[firstCorrelation[1:]].values #the rest of the columns that are not zeros

# find the scaling factors using least squares function
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name,factor) for name, factor in zip(firstCorrelation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Packet Length Std', np.float64(1.1607160816175794))
('Packet Length Variance', np.float64(160.25967924010962))
('Total Length of Fwd Packet', np.float64(6271.216232539372))
('Fwd Packet Length Mean', np.float64(0.7132122210543036))
('Fwd Packet Length Std', np.float64(1.1607160816175794))
('Fwd Segment Size Avg', np.float64(0.4210840873569066))
('Subflow Fwd Bytes', np.float64(0.8054518323891989))


In [28]:
dosDataset['Average Packet Length'] = np.random.uniform(MinMaxDict['Average Packet Length'][0]*0.85, MinMaxDict['Average Packet Length'][1]*1.15, NUM_OF_ROWS)

for index, row in dosDataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values
        # calculate the value we want to add into the dataset in the given row
        delta = random.uniform(factor * 0.1, factor * 0.2) # select a delta
        updatedFactor = factor + random.choice([-1, 1]) * delta
        dosDataset.loc[index, col] = row['Average Packet Length'] * updatedFactor

In [29]:
x = dosDataset[dosDataset['Packet Length Std'] > 114]
x[x['Packet Length Std'] < 120].tail(20)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
8602,0.0,126.755428,0.0,0.0,118.581099,16748.627659,941427.999170,0.0,74.384631,0.0,130.226061,0,0,0,0,43.771885,0,119.103517,4126,6339.0,2350,30.110472,258.756114,0.298815,0.003073,0.229221
8606,0.0,84.922279,0.0,0.0,117.640925,11866.167422,596330.782915,0.0,68.859736,0.0,81.853055,0,0,0,0,42.131203,0,78.136438,2211,4520.0,2760,35.167536,280.044487,1.802771,0.004674,0.040658
8643,0.0,85.704113,0.0,0.0,118.619074,12032.555814,470572.861331,0.0,49.174407,0.0,109.443952,0,0,0,0,30.800870,0,80.873807,2630,5313.0,731,36.625441,284.986041,1.184726,0.003788,0.008612
8645,0.0,126.724926,0.0,0.0,118.234759,17893.779221,913198.076041,0.0,73.793036,0.0,175.425231,0,0,0,0,43.255290,0,113.483642,2418,5165.0,1038,22.933151,450.315768,2.350324,0.003083,0.010766
8655,0.0,126.832089,0.0,0.0,118.338903,23215.545343,953301.568951,0.0,100.583313,0.0,120.839218,0,0,0,0,61.145295,0,87.527646,3597,5633.0,659,11.516036,898.384935,0.960344,0.001550,0.086343
8659,0.0,85.225146,0.0,0.0,114.556396,15077.985800,474323.440247,0.0,67.251828,0.0,118.614428,0,0,0,0,32.213172,0,76.906813,3232,6917.0,1282,23.188434,440.807296,15.278953,0.003130,0.188171
8669,0.0,117.533445,0.0,0.0,114.779259,15223.972809,614727.080157,0.0,96.429477,0.0,158.000093,0,0,0,0,41.418165,0,78.158330,3014,4878.0,404,37.434796,190.088701,26.292208,0.003493,0.278853
8677,0.0,122.760157,0.0,0.0,116.403254,15986.067105,861813.274903,0.0,72.436306,0.0,119.508363,0,0,0,0,41.627818,0,114.029377,3133,6789.0,1420,32.450182,216.839470,1.764683,0.003358,0.012464
8719,0.0,114.013787,0.0,0.0,116.669996,15543.317768,591504.129641,0.0,91.065208,0.0,109.902506,0,0,0,0,53.533225,0,76.022872,3668,7765.0,627,27.815842,352.154602,0.880211,0.002829,0.029989
8732,0.0,122.637193,0.0,0.0,119.630270,17179.569455,896112.679946,0.0,104.080015,0.0,169.995816,0,0,0,0,45.504097,0,112.250622,3698,7970.0,2934,15.171265,466.221001,9.533826,0.002058,0.015544


### Independant Columns

In [30]:
# finding the correlation between the Packet Length Std column to the rest of the columns in order to create new data
independantColumns = ['Packet Length Min', 'Packet Length Max', 'Fwd Packet Length Max', 'Fwd Packet Length Min']

for col in independantColumns:
    dosDataset[col] = (np.random.uniform(MinMaxDict[col][0]*0.85, MinMaxDict[col][1]*1.15, NUM_OF_ROWS)).astype(int)

dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,147.043814,57,1042,142.592761,20927.095692,1.099512e+06,1000,123.380305,15,201.204854,0,0,0,0,55.610180,0,141.817343,3577,5735.0,0,7.561949,1298.146733,0.742613,0.001023,0.057751
1,0.0,148.022099,75,1170,204.692213,28138.117225,8.143548e+05,975,90.422340,20,141.754786,0,0,0,0,52.459133,0,141.501485,1650,2562.0,0,12.779367,814.669213,1.402993,0.001221,0.104687
2,0.0,81.468583,62,1167,78.617063,10852.433469,5.630750e+05,920,52.215433,26,107.180144,0,0,0,0,38.326344,0,52.872251,1642,3531.0,0,13.117692,541.948252,0.195677,0.001370,0.004042
3,0.0,109.603949,64,849,111.580035,20398.506910,8.185441e+05,1021,88.048906,26,106.313538,0,0,0,0,37.665066,0,99.692026,2684,4242.0,0,23.037182,319.613246,15.497791,0.002183,0.014253
4,0.0,92.183830,57,731,88.979554,16744.001032,5.025448e+05,612,75.816062,19,89.043023,0,0,0,0,45.664051,0,84.610167,2201,4570.0,0,19.756791,494.909240,1.321188,0.002002,0.022750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,0.0,108.786025,58,938,111.080799,15252.369463,7.816143e+05,567,88.011452,16,102.803874,0,0,0,0,40.645804,0,76.290243,2195,4573.0,2947,21.838815,445.413498,15.004167,0.002190,0.007130
8996,0.0,156.807829,80,981,146.464903,22097.733503,8.704528e+05,1000,127.876213,17,156.797929,0,0,0,0,74.422325,0,139.871707,3657,7733.0,414,35.557013,286.941586,2.862201,0.004694,0.029678
8997,0.0,130.457710,48,1152,131.118541,18287.715062,7.298058e+05,1101,79.541692,34,177.584986,0,0,0,0,63.974095,0,86.371563,1570,2557.0,2947,20.488110,471.922151,1.058274,0.002797,0.005263
8998,0.0,77.774878,44,735,100.643571,14317.369191,4.092347e+05,1131,66.201033,17,108.006798,0,0,0,0,27.870324,0,56.365638,1695,3492.0,3032,22.115640,336.062063,13.938611,0.003061,0.008227


### Adding labels and verifiying the dataset

In [31]:
# adding number of ports and a label to the dataset
dosDataset['Number of Ports'] = np.full(shape=NUM_OF_ROWS, fill_value=1, dtype=int)
dosDataset['Label'] = ATTACK_NAME

In [32]:
dosDataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std,Label
0,1,147.043814,57,1042,142.592761,20927.095692,1.099512e+06,1000,123.380305,15,201.204854,0,0,0,0,55.610180,0,141.817343,3577,5735.0,0,7.561949,1298.146733,0.742613,0.001023,0.057751,DoS
1,1,148.022099,75,1170,204.692213,28138.117225,8.143548e+05,975,90.422340,20,141.754786,0,0,0,0,52.459133,0,141.501485,1650,2562.0,0,12.779367,814.669213,1.402993,0.001221,0.104687,DoS
2,1,81.468583,62,1167,78.617063,10852.433469,5.630750e+05,920,52.215433,26,107.180144,0,0,0,0,38.326344,0,52.872251,1642,3531.0,0,13.117692,541.948252,0.195677,0.001370,0.004042,DoS
3,1,109.603949,64,849,111.580035,20398.506910,8.185441e+05,1021,88.048906,26,106.313538,0,0,0,0,37.665066,0,99.692026,2684,4242.0,0,23.037182,319.613246,15.497791,0.002183,0.014253,DoS
4,1,92.183830,57,731,88.979554,16744.001032,5.025448e+05,612,75.816062,19,89.043023,0,0,0,0,45.664051,0,84.610167,2201,4570.0,0,19.756791,494.909240,1.321188,0.002002,0.022750,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,1,108.786025,58,938,111.080799,15252.369463,7.816143e+05,567,88.011452,16,102.803874,0,0,0,0,40.645804,0,76.290243,2195,4573.0,2947,21.838815,445.413498,15.004167,0.002190,0.007130,DoS
8996,1,156.807829,80,981,146.464903,22097.733503,8.704528e+05,1000,127.876213,17,156.797929,0,0,0,0,74.422325,0,139.871707,3657,7733.0,414,35.557013,286.941586,2.862201,0.004694,0.029678,DoS
8997,1,130.457710,48,1152,131.118541,18287.715062,7.298058e+05,1101,79.541692,34,177.584986,0,0,0,0,63.974095,0,86.371563,1570,2557.0,2947,20.488110,471.922151,1.058274,0.002797,0.005263,DoS
8998,1,77.774878,44,735,100.643571,14317.369191,4.092347e+05,1131,66.201033,17,108.006798,0,0,0,0,27.870324,0,56.365638,1695,3492.0,3032,22.115640,336.062063,13.938611,0.003061,0.008227,DoS


In [33]:
dosSamples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.0,21.0,21.0,21.0,21.000000,21.0,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,1.0,118.162351,62.000000,835.333333,137.140846,18874.235571,737972.047619,801.333333,84.162351,28.000000,137.140846,0.0,0.0,0.0,0.0,49.548964,0.0,95.038656,3010.761905,5627.904762,562.238095,15.224132,700.470398,4.264842,0.001767,0.049179
std,0.0,6.975820,5.796551,29.169048,8.363914,2241.856330,128170.354837,29.169048,6.975820,5.796551,8.363914,0.0,0.0,0.0,0.0,6.637618,0.0,9.910132,551.209389,852.917400,897.206103,7.796555,276.305968,6.595806,0.000968,0.074095
min,1.0,104.405674,54.000000,792.000000,120.419451,14500.844118,444848.000000,758.000000,70.405674,20.000000,120.419451,0.0,0.0,0.0,0.0,36.545959,0.0,74.607832,1752.000000,3778.000000,0.000000,5.743676,246.263767,0.023122,0.000893,0.001892
25%,1.0,109.976270,54.000000,818.000000,127.842838,16343.791319,640328.000000,784.000000,75.976270,20.000000,127.842838,0.0,0.0,0.0,0.0,41.808258,0.0,88.539871,2656.000000,5385.000000,0.000000,9.452635,457.067517,0.063931,0.001061,0.002862
50%,1.0,121.000527,66.000000,831.000000,140.240047,19667.270800,773385.000000,797.000000,87.000527,32.000000,140.240047,0.0,0.0,0.0,0.0,52.030510,0.0,96.074446,3234.000000,5802.000000,0.000000,12.632054,756.444383,0.137966,0.001322,0.003427
75%,1.0,123.044975,66.000000,849.000000,142.734482,20373.132473,846118.000000,815.000000,89.044975,32.000000,142.734482,0.0,0.0,0.0,0.0,53.888530,0.0,101.706229,3366.000000,6244.000000,1035.000000,19.535407,943.017495,5.236684,0.002188,0.054854
max,1.0,127.054656,66.000000,911.000000,147.443905,21739.705195,873411.000000,877.000000,93.054656,32.000000,147.443905,0.0,0.0,0.0,0.0,58.701364,0.0,117.268366,3689.000000,6834.000000,3074.000000,34.094212,1120.536707,20.601452,0.004061,0.230153


In [34]:
dosDataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9.000000e+03,9000.000000,9000.000000,9000.000000,9000.000000,9000.0,9000.0,9000.0,9000.0,9000.000000,9000.0,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000,9000.000000
mean,1.0,121.495928,61.973111,888.097889,140.776295,19458.810648,7.603852e+05,851.372111,86.594984,27.418222,141.321538,0.0,0.0,0.0,0.0,51.111657,0.0,97.776783,2861.120222,5229.870778,787.155667,22.608102,532.919539,6.006740,0.002629,0.067981
std,0.0,26.888085,13.806480,181.424764,38.286168,5293.145122,2.060619e+05,175.810195,23.418837,7.744272,38.323019,0.0,0.0,0.0,0.0,13.836250,0.0,26.292675,784.638051,1651.722496,994.274012,10.489316,385.614713,7.910678,0.001302,0.089145
min,1.0,75.451432,38.000000,572.000000,70.665976,9736.930035,3.806079e+05,547.000000,43.455284,14.000000,70.751256,0.0,0.0,0.0,0.0,25.571600,0.0,49.029240,1489.000000,2197.000000,0.000000,4.394261,170.041244,0.019663,0.000414,0.000209
25%,1.0,98.108009,50.000000,732.000000,111.224936,15294.569146,5.997333e+05,700.000000,68.386488,21.000000,111.855615,0.0,0.0,0.0,0.0,40.466608,0.0,77.197477,2187.750000,3913.000000,0.000000,13.547278,276.940191,0.741335,0.001531,0.008365
50%,1.0,121.308769,62.000000,889.000000,137.391420,19005.740784,7.456241e+05,850.000000,84.501067,27.000000,138.020576,0.0,0.0,0.0,0.0,49.979637,0.0,95.568014,2853.000000,5084.000000,47.500000,22.625177,386.616706,1.941433,0.002562,0.021624
75%,1.0,145.027784,74.000000,1043.000000,164.730340,22831.454544,8.898758e+05,1002.000000,101.278136,34.000000,165.770241,0.0,0.0,0.0,0.0,59.615675,0.0,114.358662,3535.000000,6348.250000,1572.000000,31.544778,638.536413,9.085513,0.003606,0.104833
max,1.0,168.024893,86.000000,1203.000000,233.349856,32283.031854,1.258554e+06,1159.000000,142.788521,41.000000,232.879770,0.0,0.0,0.0,0.0,84.470807,0.0,160.823526,4241.000000,9273.000000,3074.000000,41.166914,2321.145044,30.111836,0.005715,0.341234


In [35]:
dosDataset.shape

(9000, 27)

In [ ]:
# save the dataset
# dosDataset.to_csv('dos_goldeneye_dataset_new.csv', index=False)

---